In [4]:
# 测试读取 PDF 【高阶】《运行手册》HO-0907 飞行运行.pdf
import fitz  
import re

def extract_text_from_pdf1(pdf_path):
    doc = fitz.open(pdf_path)
    paragraphs = []
    chapter_regex = re.compile(r'^(\d+\.){1,2}(\D|$)')
    pass_first_n_page = 4  # 跳过前几页

    for page_num in range(len(doc)):
        # 跳过前面几页
        if pass_first_n_page>0:
            pass_first_n_page -= 1
            continue    
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"] #字典形式返回结构化数据，取文本块
        page_height = page.rect.height

        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    # 排除页眉和页尾，假设页眉在页面的前10%部分，页尾在页面的后8%部分
                    line_bbox = line["bbox"]
                    if line_bbox[1] < page_height * 0.1 or line_bbox[1] > page_height * 0.92:
                        continue
                    text = ""
                    for span in line["spans"]:
                        text += span["text"]
                    if text.strip():
                        chapter_match = chapter_regex.match(text.strip())
                        if chapter_match:
                            chapter_number = chapter_match.group(0)
                            chapter_content = text.strip()[len(chapter_number):].strip()
                            paragraphs.append((chapter_number.strip(), chapter_content))
                        else:
                            if paragraphs:
                                paragraphs[-1] = (paragraphs[-1][0], paragraphs[-1][1] + " " + text.strip())
    return paragraphs


In [5]:
# 测试读取 PDF  大型飞机公共航空运输承运人运行合格审定规则.pdf
import fitz  
import re

def extract_text_from_pdf2(pdf_path):
    doc = fitz.open(pdf_path)
    paragraphs = []
    chapter_regex = re.compile(r'第([\d\.]+) 条\s+(.*)')
    chapter_regex2 = re.compile(r' (a-z)')
    pass_first_n_page = 3  # 跳过前几页

    for page_num in range(len(doc)):
        # 跳过前面几页
        if pass_first_n_page>0:
            pass_first_n_page -= 1
            continue    
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        page_height = page.rect.height

        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    # 排除页眉和页尾，假设页眉在页面的前8%部分，页尾在页面的后8%部分
                    line_bbox = line["bbox"]
                    if line_bbox[1] < page_height * 0.08 or line_bbox[1] > page_height * 0.92:
                        continue
                    text = ""
                    for span in line["spans"]:
                        text += span["text"]
                    if text.strip():
                        chapter_match = chapter_regex.match(text.strip())
                        if chapter_match:
                            chapter_number = "第 {} 条".format(chapter_match.group(1))
                            chapter_content = text.strip()[len(chapter_number):].strip()
                            paragraphs.append((chapter_number.strip(), chapter_content))
                        else:
                            if paragraphs:
                                paragraphs[-1] = (paragraphs[-1][0], paragraphs[-1][1] + " " + text.strip())
    return paragraphs


In [6]:
pdf_path1 = 'source_pdf/【高阶】《运行手册》HO-0907 飞行运行.pdf'
extracted_paragraphs1 = extract_text_from_pdf1(pdf_path1)
print(len(extracted_paragraphs1))

# 打印提取结果
for paragraph_number, paragraph_content in extracted_paragraphs1:
    print(f"片段编号: {paragraph_number}")
    print(f"片段内容: {paragraph_content}")
    print("-------------------------------------")
    print()

295
片段编号: 1.
片段内容: 飞行机组在开始滑行、进入滑行道和跑道、穿越滑行道和跑道以及起飞和着陆都必 须得到空中交通管制相应的许可。
-------------------------------------

片段编号: 2.
片段内容: 标准操作程序
-------------------------------------

片段编号: 2.1.
片段内容: 公司根据各机型手册，结合自身运行特点，为每个机型建立了标准操作程序；
-------------------------------------

片段编号: 2.2.
片段内容: 飞行机组应按照公司制定的标准操作程序操纵飞机；
-------------------------------------

片段编号: 2.3.
片段内容: 如需偏离标准操作程序， 飞行机组必须进行充分的沟通， 包括与其他相关机组 人员、 地面人员以及空中交通管制员等的沟通， 并评估偏离标准操作程序可能 造成的风险。
-------------------------------------

片段编号: 3.
片段内容: 检查单的使用
-------------------------------------

片段编号: 3.1.
片段内容: 公司为每一机型提供经批准的检查单， 并配备到公司运行的每架飞机上飞行机 组可以快速取用的位置；
-------------------------------------

片段编号: 3.2.
片段内容: 飞行机组在运行过程中，必须使用公司配备的经批准的检查单；
-------------------------------------

片段编号: 3.3.
片段内容: 除规定的记忆项目外，任何人不得背诵检查单；
-------------------------------------

片段编号: 3.4.
片段内容: 执行检查单必须使用标准用语；
-------------------------------------

片段编号: 3.5.
片段内容: 执行检查单过程中，如果某项目不能立即执行或PF 未应答时，检查单必须停 止在此项目位置，直到完成该项目或遗漏、错误得到纠正后，方可继续执行检 查单剩余项目；
-------------------------

In [7]:
pdf_path2 = 'source_pdf/大型飞机公共航空运输承运人运行合格审定规则.pdf'
extracted_paragraphs2 = extract_text_from_pdf2(pdf_path2)
print(len(extracted_paragraphs2))

# 打印提取结果
for paragraph_number, paragraph_content in extracted_paragraphs2:
    print(f"片段编号: {paragraph_number}")
    print(f"片段内容: {paragraph_content}")
    print("-------------------------------------")
    print()


301
片段编号: 第 121.1 条
片段内容: 目的和依据 为了对大型飞机公共航空运输承运人进行运行合格审 定和持续监督检查，保证其达到并保持规定的运行安全水 平，根据《中华人民共和国民用航空法》和《国务院对确需 保留的行政审批项目设定行政许可的决定》 ，制定本规则。
-------------------------------------

片段编号: 第 121.3 条
片段内容: 适用范围 (a)本规则适用于在中华人民共和国境内依法设立的航 空运营人实施的下列公共航空运输运行： (1)使用最大起飞重量超过5,700 千克的多发涡轮驱动飞 机实施的定期载客运输飞行； (2)使用旅客座位数超过30 座或者最大商载超过3,400 千克的多发涡轮驱动飞机实施的不定期载客运输飞行； (3)使用最大商载超过3,400 千克的多发涡轮驱动飞机实 施的全货物运输飞行。 (b)对于适用于本条(a)款规定的航空运营人， 在本规则中 称之为大型飞机公共航空运输承运人。 (c)对于应当按照本规则审定合格的大型飞机公共航空 运输承运人，中国民用航空局(以下简称中国民航局)和民用 航空地区管理局（以下简称民航地区管理局）按照审定情况 在其运行合格证及其运行规范中批准其实施下列一项或者 多项运行种类的运行： (1)国内定期载客运行，是指符合本条(a)款第(1)项规定， 在中华人民共和国境内两点之间的运行，或者一个国内地点 与另一个由局方专门指定、视为国内地点的国外地点之间的 运行； (2)国际定期载客运行，是指符合本条(a)款第(1)项规定， 在一个国内地点和一个国外地点之间，两个国外地点之间， 或者一个国内地点与另一个由局方专门指定、视为国外地点 的国内地点之间的运行； (3)补充运行，是指符合本条(a)款第(2)项、第(3)项规定 的运行。 (d)大型飞机公共航空运输承运人应当遵守其他有关的 涉及民航管理的规章，但在本规则对相应要求进行了增补或 者提出了更高标准的情况下，应当按照本规则的要求执行。 (e)大型飞机公共航空运输承运人在运行中所使用的人 员和大型飞机公共航空运输承运人所载运的人员应当遵守 本规则中的适用要求。 (f)在本规则中，对于载运邮件的飞行，视为载运货物飞 行；对于同时载运旅客和货物的飞行，视为载运旅客飞行， 但应当同时满足本规则中有关货物运输条款的要求。
--

In [8]:
paragraph_dict1 = {}
key_prefix1 = 'file1'

for i in range(len(extracted_paragraphs1)):
    key = f'{key_prefix1}_{i}_{extracted_paragraphs1[i][0]}'.strip()
    value = extracted_paragraphs1[i][1].strip()
    paragraph_dict1[key] = value

print(len(paragraph_dict1))

295


In [9]:
paragraph_dict1

{'file1_0_1.': '飞行机组在开始滑行、进入滑行道和跑道、穿越滑行道和跑道以及起飞和着陆都必 须得到空中交通管制相应的许可。',
 'file1_1_2.': '标准操作程序',
 'file1_2_2.1.': '公司根据各机型手册，结合自身运行特点，为每个机型建立了标准操作程序；',
 'file1_3_2.2.': '飞行机组应按照公司制定的标准操作程序操纵飞机；',
 'file1_4_2.3.': '如需偏离标准操作程序， 飞行机组必须进行充分的沟通， 包括与其他相关机组 人员、 地面人员以及空中交通管制员等的沟通， 并评估偏离标准操作程序可能 造成的风险。',
 'file1_5_3.': '检查单的使用',
 'file1_6_3.1.': '公司为每一机型提供经批准的检查单， 并配备到公司运行的每架飞机上飞行机 组可以快速取用的位置；',
 'file1_7_3.2.': '飞行机组在运行过程中，必须使用公司配备的经批准的检查单；',
 'file1_8_3.3.': '除规定的记忆项目外，任何人不得背诵检查单；',
 'file1_9_3.4.': '执行检查单必须使用标准用语；',
 'file1_10_3.5.': '执行检查单过程中，如果某项目不能立即执行或PF 未应答时，检查单必须停 止在此项目位置，直到完成该项目或遗漏、错误得到纠正后，方可继续执行检 查单剩余项目；',
 'file1_11_3.6.': 'PF 必须喊出检查单的全称，PM 应在检查单完成后宣布“XXX 检查单完成”；',
 'file1_12_3.7.': '对于正常检查单的使用方法，具体详见机型SOP。',
 'file1_13_4.': '交叉检查',
 'file1_14_4.1.': '交叉检查是预防发生人为差错、及时发现系统故障的重要手段。在操纵飞机、 操作设备时，PF 和PM 应共同证实；特别是涉及到飞机关键设备（动力、液 压、电源、增压等）、飞机构型改变（襟/缝翼、起落架、减速板）、航路/航 向/高度/速度变化、计算机重置、航空器控制的交接、重要飞行数据的计算和 输入等时，飞机无论是正常、非正常、紧急情况下，均应互相证实后方可进行 操作；',
 'file1_15_4.2.': '飞行过程中，飞行机组在操作发动机主控电门、推力手柄、灭火手柄、I

In [10]:
paragraph_dict2 = {}
key_prefix2 = 'file2'

for i in range(len(extracted_paragraphs2)):
    key = f'{key_prefix2}_{i}_{extracted_paragraphs2[i][0]}'.strip()
    value = extracted_paragraphs2[i][1].strip()
    paragraph_dict2[key] = value

print(len(paragraph_dict2))

301


In [11]:
paragraph_dict2

{'file2_0_第 121.1 条': '目的和依据 为了对大型飞机公共航空运输承运人进行运行合格审 定和持续监督检查，保证其达到并保持规定的运行安全水 平，根据《中华人民共和国民用航空法》和《国务院对确需 保留的行政审批项目设定行政许可的决定》 ，制定本规则。',
 'file2_1_第 121.3 条': '适用范围 (a)本规则适用于在中华人民共和国境内依法设立的航 空运营人实施的下列公共航空运输运行： (1)使用最大起飞重量超过5,700 千克的多发涡轮驱动飞 机实施的定期载客运输飞行； (2)使用旅客座位数超过30 座或者最大商载超过3,400 千克的多发涡轮驱动飞机实施的不定期载客运输飞行； (3)使用最大商载超过3,400 千克的多发涡轮驱动飞机实 施的全货物运输飞行。 (b)对于适用于本条(a)款规定的航空运营人， 在本规则中 称之为大型飞机公共航空运输承运人。 (c)对于应当按照本规则审定合格的大型飞机公共航空 运输承运人，中国民用航空局(以下简称中国民航局)和民用 航空地区管理局（以下简称民航地区管理局）按照审定情况 在其运行合格证及其运行规范中批准其实施下列一项或者 多项运行种类的运行： (1)国内定期载客运行，是指符合本条(a)款第(1)项规定， 在中华人民共和国境内两点之间的运行，或者一个国内地点 与另一个由局方专门指定、视为国内地点的国外地点之间的 运行； (2)国际定期载客运行，是指符合本条(a)款第(1)项规定， 在一个国内地点和一个国外地点之间，两个国外地点之间， 或者一个国内地点与另一个由局方专门指定、视为国外地点 的国内地点之间的运行； (3)补充运行，是指符合本条(a)款第(2)项、第(3)项规定 的运行。 (d)大型飞机公共航空运输承运人应当遵守其他有关的 涉及民航管理的规章，但在本规则对相应要求进行了增补或 者提出了更高标准的情况下，应当按照本规则的要求执行。 (e)大型飞机公共航空运输承运人在运行中所使用的人 员和大型飞机公共航空运输承运人所载运的人员应当遵守 本规则中的适用要求。 (f)在本规则中，对于载运邮件的飞行，视为载运货物飞 行；对于同时载运旅客和货物的飞行，视为载运旅客飞行， 但应当同时满足本规则中有关货物运输条款的要求。',
 'file2_2_第 121.5 条': '定义 (a)在本规则中， 局方是

In [12]:
# 民航总局文件拆分到a/b/c粒度
def split_paragraph_dict_lv2(paragraph_dict):
    res = {}
    for src_key in paragraph_dict:
        src_content = paragraph_dict[src_key]
        
        # 先提取所有的"(字母)"次标题
        sub_titles = re.findall(r'\s+\(([a-z])\)', src_content)
        # 提取主标题, 第一个次标题前
        first_sub_titles_start = src_content.find(f"({sub_titles[0]})", 0) if len(sub_titles) > 0 else len(src_content)
        main_title = src_content[:first_sub_titles_start]
        # 初始化内容列表和标题计数器
        contents = []
        title_counter = 1
        # 根据次标题位置分割文本，提取内容
        start_pos = 0
        for sub_title in sub_titles:
            # 查找当前次标题的位置
            title_start = src_content.find(f" ({sub_title})", start_pos)
            # 查找下一个次标题或文本结束的位置
            if title_counter < len(sub_titles):
                next_title = sub_titles[title_counter]
                next_title_start = src_content.find(f" ({next_title})", title_start)
                content = src_content[title_start+len(f" ({sub_title})"):next_title_start].strip()
            else:
                # 如果是最后一个次标题，则提取到字符串末尾
                content = src_content[title_start+len(f" ({sub_title})"):].strip()
            contents.append(content)
            start_pos = title_start + len(f"({sub_title})")
            title_counter += 1
            
        # 生成最终的字典
        # 如果不能被拆分(a), (b)...
        if len(sub_titles) == 0 or len(contents) == 0:
            res[src_key] = src_content
        else:
            split_keys = [f"{src_key}_({sub_title})" for sub_title in sub_titles]
            split_values = [f"{main_title}{content}" for content in contents]
            for sub_key, sub_value in zip(split_keys, split_values):
                res[sub_key] = sub_value
    return res

In [13]:
paragraph_dict2_lv2 = split_paragraph_dict_lv2(paragraph_dict2)

In [14]:
len(paragraph_dict2_lv2)

1032

In [15]:
paragraph_dict2_lv2

{'file2_0_第 121.1 条': '目的和依据 为了对大型飞机公共航空运输承运人进行运行合格审 定和持续监督检查，保证其达到并保持规定的运行安全水 平，根据《中华人民共和国民用航空法》和《国务院对确需 保留的行政审批项目设定行政许可的决定》 ，制定本规则。',
 'file2_1_第 121.3 条_(a)': '适用范围 本规则适用于在中华人民共和国境内依法设立的航 空运营人实施的下列公共航空运输运行： (1)使用最大起飞重量超过5,700 千克的多发涡轮驱动飞 机实施的定期载客运输飞行； (2)使用旅客座位数超过30 座或者最大商载超过3,400 千克的多发涡轮驱动飞机实施的不定期载客运输飞行； (3)使用最大商载超过3,400 千克的多发涡轮驱动飞机实 施的全货物运输飞行。',
 'file2_1_第 121.3 条_(b)': '适用范围 对于适用于本条(a)款规定的航空运营人， 在本规则中 称之为大型飞机公共航空运输承运人。',
 'file2_1_第 121.3 条_(c)': '适用范围 对于应当按照本规则审定合格的大型飞机公共航空 运输承运人，中国民用航空局(以下简称中国民航局)和民用 航空地区管理局（以下简称民航地区管理局）按照审定情况 在其运行合格证及其运行规范中批准其实施下列一项或者 多项运行种类的运行： (1)国内定期载客运行，是指符合本条(a)款第(1)项规定， 在中华人民共和国境内两点之间的运行，或者一个国内地点 与另一个由局方专门指定、视为国内地点的国外地点之间的 运行； (2)国际定期载客运行，是指符合本条(a)款第(1)项规定， 在一个国内地点和一个国外地点之间，两个国外地点之间， 或者一个国内地点与另一个由局方专门指定、视为国外地点 的国内地点之间的运行； (3)补充运行，是指符合本条(a)款第(2)项、第(3)项规定 的运行。',
 'file2_1_第 121.3 条_(d)': '适用范围 大型飞机公共航空运输承运人应当遵守其他有关的 涉及民航管理的规章，但在本规则对相应要求进行了增补或 者提出了更高标准的情况下，应当按照本规则的要求执行。',
 'file2_1_第 121.3 条_(e)': '适用范围 大型飞机公共航空运输承运人在运行中所使用的人 员和大型飞机公共航空运输承运人所载运的人员应当遵守 本规则中的

In [16]:
# 民航总局文件 进一步 拆分到1/2/3粒度
def split_paragraph_dict_lv3(paragraph_dict2_lv2):
    res = {}
    for src_key in paragraph_dict2_lv2:
        src_content = paragraph_dict2_lv2[src_key]
        
        # 先提取所有的"(数字)"次标题
        sub_titles = re.findall(r"\s+\(([0-9]+)\)", src_content)
        # print(sub_titles)
        # 提取主标题, 第一个次标题前
        first_sub_titles_start = src_content.find(f"({sub_titles[0]})", 0) if len(sub_titles) > 0 else len(src_content)
        main_title = src_content[:first_sub_titles_start]
        # 初始化内容列表和标题计数器
        contents = []
        title_counter = 1
        # 根据次标题位置分割文本，提取内容
        start_pos = 0
        for sub_title in sub_titles:
            # 查找当前次标题的位置
            title_start = src_content.find(f" ({sub_title})", start_pos)
            # 查找下一个次标题或文本结束的位置
            if title_counter < len(sub_titles):
                next_title = sub_titles[title_counter]
                next_title_start = src_content.find(f" ({next_title})", title_start)
                content = src_content[title_start+len(f" ({sub_title})"):next_title_start].strip()
            else:
                # 如果是最后一个次标题，则提取到字符串末尾
                content = src_content[title_start+len(f" ({sub_title})"):].strip()
            contents.append(content)
            start_pos = title_start + len(f"({sub_title})")
            title_counter += 1
            
        # 生成最终的字典
        # 如果不能被拆分(1), (2)...
        if len(sub_titles) == 0 or len(contents) == 0:
            res[src_key] = src_content
        else:
            split_keys = [f"{src_key}_({sub_title})" for sub_title in sub_titles]
            split_values = [f"{main_title}{content}" for content in contents]
            for sub_key, sub_value in zip(split_keys, split_values):
                res[sub_key] = sub_value
    return res

In [17]:
paragraph_dict2_lv3 = split_paragraph_dict_lv3(paragraph_dict2_lv2)

In [18]:
len(paragraph_dict2_lv3)

1819

In [19]:
paragraph_dict2_lv3

{'file2_0_第 121.1 条': '目的和依据 为了对大型飞机公共航空运输承运人进行运行合格审 定和持续监督检查，保证其达到并保持规定的运行安全水 平，根据《中华人民共和国民用航空法》和《国务院对确需 保留的行政审批项目设定行政许可的决定》 ，制定本规则。',
 'file2_1_第 121.3 条_(a)_(1)': '适用范围 本规则适用于在中华人民共和国境内依法设立的航 空运营人实施的下列公共航空运输运行： 使用最大起飞重量超过5,700 千克的多发涡轮驱动飞 机实施的定期载客运输飞行；',
 'file2_1_第 121.3 条_(a)_(2)': '适用范围 本规则适用于在中华人民共和国境内依法设立的航 空运营人实施的下列公共航空运输运行： 使用旅客座位数超过30 座或者最大商载超过3,400 千克的多发涡轮驱动飞机实施的不定期载客运输飞行；',
 'file2_1_第 121.3 条_(a)_(3)': '适用范围 本规则适用于在中华人民共和国境内依法设立的航 空运营人实施的下列公共航空运输运行： 使用最大商载超过3,400 千克的多发涡轮驱动飞机实 施的全货物运输飞行。',
 'file2_1_第 121.3 条_(b)': '适用范围 对于适用于本条(a)款规定的航空运营人， 在本规则中 称之为大型飞机公共航空运输承运人。',
 'file2_1_第 121.3 条_(c)_(1)': '适用范围 对于应当按照本规则审定合格的大型飞机公共航空 运输承运人，中国民用航空局(以下简称中国民航局)和民用 航空地区管理局（以下简称民航地区管理局）按照审定情况 在其运行合格证及其运行规范中批准其实施下列一项或者 多项运行种类的运行： 国内定期载客运行，是指符合本条(a)款第(1)项规定， 在中华人民共和国境内两点之间的运行，或者一个国内地点 与另一个由局方专门指定、视为国内地点的国外地点之间的 运行；',
 'file2_1_第 121.3 条_(c)_(2)': '适用范围 对于应当按照本规则审定合格的大型飞机公共航空 运输承运人，中国民用航空局(以下简称中国民航局)和民用 航空地区管理局（以下简称民航地区管理局）按照审定情况 在其运行合格证及其运行规范中批准其实施下列一项或者 多项运行种类的运行： 国际定期载客运行，是指符合本条(a)款第(1)项

In [20]:
# 保存每个片段到字典文件 
import pickle

def save_dict_to_file(dict, file_path):
    with open(file_path, 'wb') as pickle_file:
        pickle.dump(dict, pickle_file)

In [21]:
save_dict_to_file(paragraph_dict1, '【高阶】《运行手册》HO-0907 飞行运行.pkl')
save_dict_to_file(paragraph_dict2, '大型飞机公共航空运输承运人运行合格审定规则.pkl')

In [22]:
save_dict_to_file(paragraph_dict2_lv2, '大型飞机公共航空运输承运人运行合格审定规则 lv2.pkl')
save_dict_to_file(paragraph_dict2_lv3, '大型飞机公共航空运输承运人运行合格审定规则 lv3.pkl')

In [23]:
# 测试读取定级部分的字典文件
import pickle
def read_dict_file(file_path):
    with open(file_path, 'rb') as f:
            data = pickle.load(f)
    return data

In [24]:
# 读取文本片段字典文件，每个文本片段调用zhipu Embedding，存储到数据库（本地pkl文件）
text_dict1 = read_dict_file('【高阶】《运行手册》HO-0907 飞行运行.pkl')
text_dict2 = read_dict_file('大型飞机公共航空运输承运人运行合格审定规则.pkl')

In [25]:
text_dict2_lv2 = read_dict_file('大型飞机公共航空运输承运人运行合格审定规则 lv2.pkl')
text_dict2_lv3 = read_dict_file('大型飞机公共航空运输承运人运行合格审定规则 lv3.pkl')

In [26]:
print(len(text_dict1))
print(len(text_dict2))

295
301


In [27]:
print(len(text_dict2_lv2))
print(len(text_dict2_lv3))

1032
1819


In [31]:
from zhipuai import ZhipuAI
#调用智谱AI的embedding模型
# 智谱AI Embedding长度设置为最大4096个汉字
def get_text_embedding_from_zhipu_api(input_str, max_text_length = 4096):
    input_text = input_str.replace("\n", " ")[:max_text_length] #换行符替换为空格
    client = ZhipuAI(api_key="0e059a59be0f37e58bbafe360f663b21.7G1ccHB6WZMFvfwu") 
    response = client.embeddings.create(
        model="embedding-2", #填写需要调用的模型名称
        input=input_text,
    )
    return response.data[0].embedding


In [32]:
# 获取PDF文件所有片段的embd
def get_paragraphs_embd(dict):
    embd_dict = {}
    for key in dict:
        text = dict[key].strip() if dict[key] is not None else ''
        if text != '':
            embd = get_text_embedding_from_zhipu_api(text)
            embd_dict[key] = embd
    return embd_dict

In [33]:
embd_dict1 = get_paragraphs_embd(text_dict1)
embd_dict2 = get_paragraphs_embd(text_dict2)

In [34]:
print(len(embd_dict1))
print(len(embd_dict2))

295
301


In [35]:
embd_dict2_lv2 = get_paragraphs_embd(text_dict2_lv2)

In [36]:
print(len(embd_dict2_lv2))

1032


In [37]:
# 保存embd到数据库（pkl文件）(key： (text, embd) )
import pickle
import os

def save_paragraphs_embd_into_db(dict_text, dict_embd, embd_db_file):
    # 读取数据库已有全量embd, 需判断是否第一次保存（文件不存在）
    if os.path.exists(embd_db_file):
        with open(embd_db_file, 'rb') as f:
                all_embd_data = pickle.load(f)
    else:
        all_embd_data = {}
    # 更新数据
    for key in dict_text:
        if key in dict_embd:
            all_embd_data[key] = (dict_text[key], dict_embd[key])
        else:
            all_embd_data[key] = (dict_text[key], [])
    # 保存新的embd
    save_dict_to_file(all_embd_data, embd_db_file)

In [77]:
save_paragraphs_embd_into_db(text_dict1, embd_dict1, 'pdf_paragraphs_embd_db.pkl')
save_paragraphs_embd_into_db(text_dict2, embd_dict2, 'pdf_paragraphs_embd_db.pkl')

In [101]:
save_paragraphs_embd_into_db(text_dict2_lv2, embd_dict2_lv2, 'pdf_paragraphs_embd_db.pkl')

In [102]:
# 验证数据库内 embd
with open('pdf_paragraphs_embd_db.pkl', 'rb') as f:
    embd_db_data = pickle.load(f)
print(len(embd_db_data))
print(embd_db_data['file2_1_第 121.3 条_(a)'])

1569
('适用范围 本规则适用于在中华人民共和国境内依法设立的航 空运营人实施的下列公共航空运输运行： (1)使用最大起飞重量超过5,700 千克的多发涡轮驱动飞 机实施的定期载客运输飞行； (2)使用旅客座位数超过30 座或者最大商载超过3,400 千克的多发涡轮驱动飞机实施的不定期载客运输飞行； (3)使用最大商载超过3,400 千克的多发涡轮驱动飞机实 施的全货物运输飞行。', [0.034766708, 0.0060246247, -0.0039531603, -0.024886195, 0.03607047, 0.05136239, -0.027525235, 0.022351911, -0.03745202, 0.0457233, 0.00023352743, 0.025826212, -0.01571903, 0.006077772, 0.018617941, -0.01700733, -0.0073379893, -0.00041809038, -0.005350839, -0.01878856, 0.045124233, -0.0442488, 0.043247126, -0.009440781, -0.04921455, 0.0061606094, -0.012423283, 0.008467235, 0.032417953, -0.045706388, 0.018225458, 0.008091232, -0.033531107, 0.01005455, 0.044656187, 0.009874101, -0.011519151, 0.0046054367, 0.05641792, -0.059756063, 0.006692004, 0.02205091, -0.007660741, -0.012934246, 0.016211186, 0.019777104, 0.04020172, 0.032609895, -0.015292706, -0.0047163437, -0.026793092, 0.01353479, 0.017994126, -0.060111754, 0.01684339, 0.032023605, 0.027072353, 0.019292435, -0.037079293, -0.0066958154, -0.

In [103]:
# 计算两个embd相似度
import pickle
import numpy as np

def cal_similarity(embd1, embd2):
    score = 0
    if len(embd1) != 0 and len(embd2) != 0:
        score = np.dot(embd1, embd2) / (np.linalg.norm(embd1) * np.linalg.norm(embd2))
    return score

In [104]:
# 上传PDF 每个片段 和 数据库内所有片段 计算相似度
def cal_paragraphs_similarity(uploaded_pdf_paragraphs_dict, db_data):
    res = {}
    for key in uploaded_pdf_paragraphs_dict:
        if key not in db_data: # 上传的PDF，拆分后 key 不在数据库（一般不会发生，以防万一，可能第二次拆分PDF片段出现BUG，导致新的key出现）
                continue
        for db_key in db_data:
            if key.split('_')[0] == db_key.split('_')[0]: # 同一个PDF 自己不和自己比较
                continue
            if len(db_data[key][1]) > 0 and len(db_data[db_key][1]) > 0: # 两个片段都能找到有效embd
                simi_score = cal_similarity(db_data[key][1], db_data[db_key][1])
                if simi_score > 0:
                    if key in res:
                        res[key].append((db_key, simi_score))
                    else:
                        res[key] = [(db_key, simi_score)]
    return res

In [105]:
simi_result = cal_paragraphs_similarity(text_dict1, embd_db_data)

In [106]:
len(simi_result)

295

In [107]:
len(simi_result['file1_0_1.'])

1274

In [108]:
simi_result['file1_0_1.']

[('file2_0_第 121.1 条', np.float64(0.6092292734149538)),
 ('file2_1_第 121.3 条', np.float64(0.5860508299987357)),
 ('file2_2_第 121.5 条', np.float64(0.5129576214130442)),
 ('file2_3_第 121.7 条', np.float64(0.5877782049413514)),
 ('file2_4_第 121.9 条', np.float64(0.578726250407278)),
 ('file2_5_第 121.11 条', np.float64(0.6069513758483304)),
 ('file2_6_第 121.20 条', np.float64(0.5753701762587126)),
 ('file2_7_第 121.21 条', np.float64(0.5757351728492942)),
 ('file2_8_第 121.23 条', np.float64(0.5860056795114403)),
 ('file2_9_第 121.25 条', np.float64(0.5825928217897568)),
 ('file2_10_第 121.27 条', np.float64(0.49168077741643185)),
 ('file2_11_第 121.29 条', np.float64(0.6385049629220981)),
 ('file2_12_第 121.31 条', np.float64(0.5385774247594263)),
 ('file2_13_第 121.33 条', np.float64(0.4679486098788553)),
 ('file2_14_第 121.35 条', np.float64(0.611677275508665)),
 ('file2_15_第 121.37 条', np.float64(0.4569060260744667)),
 ('file2_16_第 121.41 条', np.float64(0.6328976871049151)),
 ('file2_17_第 121.42 条', np.fl

In [109]:
# 选取Top5相似度, 且 相似度分数超过阈值
def get_top_n_simi(simi_res, n, threshold=0):
    top5_simi_res = {}
    for k in simi_res:
        tmp = [tup for tup in simi_res[k] if tup[1] >= threshold] # 超过阈值
        if len(tmp) > 0:
            top5_simi_res[k] = sorted(tmp, key=lambda x: x[1], reverse=True)[:n] # top N
    return top5_simi_res

In [110]:
N = 5
simi_threshold = 0.75 # 阈值经过探索，设置为0.75
top_simi_result = get_top_n_simi(simi_result, N, simi_threshold)

In [111]:
len(top_simi_result)

105

In [112]:
top_simi_result

{'file1_4_2.3.': [('file2_252_第 121.645 条_(b)',
   np.float64(0.7568556627714138)),
  ('file2_208_第 121.558 条_(b)', np.float64(0.7524038057288005))],
 'file1_6_3.1.': [('file2_241_第 121.626 条_(a)',
   np.float64(0.7612133615855393)),
  ('file2_41_第 121.125 条_(i)', np.float64(0.7608692729072081))],
 'file1_7_3.2.': [('file2_74_第 121.315 条_(c)', np.float64(0.7796067931480871)),
  ('file2_252_第 121.645 条_(b)', np.float64(0.7650739153633003)),
  ('file2_74_第 121.315 条_(a)', np.float64(0.7615417012539096)),
  ('file2_74_第 121.315 条', np.float64(0.7573186895695658)),
  ('file2_149_第 121.463 条_(b)', np.float64(0.7564717334011755))],
 'file1_15_4.2.': [('file2_196_第 121.543 条_(c)',
   np.float64(0.7604449879889185)),
  ('file2_196_第 121.543 条_(b)', np.float64(0.7510546607625672))],
 'file1_30_8.2.': [('file2_194_第 121.539 条_(a)',
   np.float64(0.8448445016424574)),
  ('file2_194_第 121.539 条_(d)', np.float64(0.8210113699547629)),
  ('file2_194_第 121.539 条_(b)', np.float64(0.8184732490636305)),


In [90]:
# # # 保存所有相似的文本到本地 进一步分析相似度阈值该设置多少
# import pandas as pd

# def save_simi_text_to_excel(top_n_simi_res, db_data, file_path):
#     paired_data = []
#     for key1 in top_n_simi_res:
#         for tup in top_n_simi_res[key1]:
#             key2 = tup[0]
#             score = tup[1]
#             if key1 in db_data and key2 in db_data:
#                 pair_row = (key1, key2, score, db_data[key1][0], db_data[key2][0])
#                 paired_data.append(pair_row)
#     paired_df = pd.DataFrame(paired_data, 
#                      columns=['upload_pdf_paragraph_key', 'db_stored_paragraph_key', 'simi_score', 'upload_pdf_paragraph', 'db_stored_paragraph'])
#     paired_df.to_excel(file_path)

In [91]:
# save_simi_text_to_excel(top_simi_result, embd_db_data, 'paired_paragraphs_0.0.xlsx')

In [92]:
# test_text1 = ('input1','除为避开其他航空器或者为改变飞行高度需要偏离航线的机动飞行外， 在任何 其他航线上，飞行机组应沿该航线的导航设施或者定位点之间的连线飞行。 （空白） 设备的使用')
# test_text2_list=[
#     ('output1', '通信和导航设施 (a)对于定期载客运行， 除本条(b)款规定外， 在每次飞行 前，只有确认在航路批准时本规则第121.97 条和第121.101 条所要求的通信设备和导航设施处于良好工作状态，方可以 签派飞机在该航路或者航段上飞行。 (b)对于定期载客运行， 如果由于超出合格证持有人控制 能力的技术原因或者其他原因，在航路上没有本规则第 121.97 条和第121.101 条所要求的设施或者设备不可用，只 要机长和飞行签派员认为现有的设施与航路所要求的通信 和导航设施等同并处于良好的工作状态，即可签派飞机在相 应航路或者航段上飞行。 (c)对于补充运行，只有当通信设备和导航设施满足本规 则第121.121 条规定时，方可以放行飞机。'),
# ]

In [93]:
# test_text1 = ('input1','这个周末我会去骑行游玩')
# test_text2_list=[
#     ('output1', 'XXXXXXXX'),
#     ('output2', 'YYYYYY'),
#     ('output3', 'XXXXX')
# ]

In [94]:
# get_top1_from_top5(test_text1, test_text2_list)

In [95]:
def get_simi_id_from_chat_answer(answer_str):
    target_line = '********************'
    simi_id_line = ''
    lines = answer_str.split('\n')
    for i, line in enumerate(lines):
        if line.strip() == target_line and i + 1 < len(lines):  # 确保不是最后一行
            simi_id_line = lines[i + 1].strip()
            break
    return simi_id_line
    

In [113]:
# 对于匹配好的top5相似度片段，生成prompt，确认最终的top1

from zhipuai import ZhipuAI

# 参数tuple格式， (key, text_content)
def get_top1_from_top5(text1_tuple, text2_tuple_list):
    client = ZhipuAI(api_key="0e059a59be0f37e58bbafe360f663b21.7G1ccHB6WZMFvfwu")

    prompt_str = ""
    for i in range(len(text2_tuple_list)):
        line = f"{i+1}. ID='{text2_tuple_list[i][0]}', content='{text2_tuple_list[i][1]}' ;\n"
        prompt_str = prompt_str + line
    # print(prompt_str)
    
    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=[
            {"role": "system", "content": "你是一名中国民用航空部门法规制定者，非常熟悉中国民用航空的规章制度，接下来我需要你根据 [输入规章A] ，判断该规章制度的内容与 [另外一些规章B] 中的哪一条含义类似"},
            {"role": "user", "content": """
                # 输入规章A: ID='I01', content='在飞行的关键阶段，全体机组成员不得从事飞机安全运行所必需的工作之外的任何其他工作，机组任何成员也不得承担这些工作。'
                # 另外一些规章B: 
                    1. ID='L01', content='机组成员的值勤要求 (a)在飞行的关键阶段，合格证持有人不得要求飞行机组 成员完成飞机安全运行所必需的工作之外的任何其他工作， 飞行机组任何成员也不得承担这些工作。';
                    2. ID='L02', content='在驾驶舱值勤的每个飞行机组必 需成员，在飞行过程中应当坐在指定的值勤位置并系好安全 带；在起飞和着陆过程中应当坐在指定的值勤位置并系好安全带和肩带';
                # 问题: 需要你从 [另外一些规章B] 中挑选出与 [输入规章A] content最为相似的那一个
                # 要求: 如果你找到相似的内容，严格按照下面的格式输出对应的ID；如果你认为没有找到请输出'未找到相似内容'
                # 输出: 
                ********************
                L01
                """},
            {"role": "assistant", "content": "当然可以，我可以通过语义分析、计算文本之间的相似度，来帮助你找到最相似的文本。"},
            {"role": "user", "content": f"""
                # 输入规章A: ID='{text1_tuple[0]}', content='{text1_tuple[1]}'
                # 另外一些规章B: 
                {prompt_str}
                # 问题: 需要你从 [另外一些规章B] 中挑选出与 [输入规章A] content最为相似的那一个
                # 要求: 
                    1. 如果你找到相似的content，严格按照下面的[输出] 输出ID, 不需要输出任何其他内容, 输出: 
                ********************
                ID  
                    2. 如果你认为没有找到相似的内容，请直接输出'未找到相似内容', 不需要输出任何其他内容, 输出:
                ********************
                未找到相似内容
            """},
        ],
    )
    answer_str = response.choices[0].message.content
    print(answer_str)
    most_simi_id = get_simi_id_from_chat_answer(answer_str)
    return most_simi_id

In [114]:
# 给所有达到相似度阈值的pair，生成最终的top1 pair
def get_top1_simi_result(top_n_simi_res, db_data):
    res = []
    for key1 in top_n_simi_res:
        text1 = db_data[key1][0]
        if len(top_n_simi_res[key1]) == 1: # 只有top1没有topN
            key2 = top_n_simi_res[key1][0][0]
            simi_score = top_n_simi_res[key1][0][1]
            text2 = db_data[key2][0]
            res.append((key1, text1, key2, text2, simi_score))
            continue
        text2_tuple_list = []
        for tup in top_n_simi_res[key1]:
            key2 = tup[0]
            simi_score = tup[1]
            text2 = db_data[key2][0]
            text2_tuple_list.append((key2, text2, simi_score))
        text1_tup = (key1, text1)
        most_simi_key2 = get_top1_from_top5(text1_tup, text2_tuple_list)
        #prompt 返回的最相似的key2需要验证一下
        if most_simi_key2 in [tup[0] for tup in top_n_simi_res[key1]]:
            res.append((key1, text1, most_simi_key2, db_data[most_simi_key2][0], simi_score))
    return res
    

In [115]:
top1_simi_result = get_top1_simi_result(top_simi_result, embd_db_data)

********************
file2_208_第 121.558 条_(b)

输入规章A强调的是在偏离标准操作程序时需要进行充分的沟通和风险评估。在规章B中，file2_208_第 121.558 条_(b)提到了在紧急情况下需要通知机长，并确保理解机长的决断，这与输入规章A中提到的沟通和评估风险有相似之处。而file2_252_第 121.645 条_(b)虽然提到了在特定设备失效时遵循经批准的程序，但并没有强调沟通和风险评估，因此与输入规章A的内容不如file2_208_相似。
********************
未找到相似内容

输入规章A提到的是关于检查单的配备和可用性，而规章B中的内容分别涉及机长获取飞行前信息的要求和飞行跟踪系统的功能，这两者与输入规章A的内容并不相似。
********************
file2_74_第 121.315 条_(c)
********************
未找到相似内容

输入规章A强调的是在进行某些关键操作时，必须由另一名飞行机组人员确认后才能进行。而规章B中的内容主要涉及谁被允许控制操纵装置，并没有直接提到需要另一名机组人员确认的操作步骤。因此，根据提供的信息，没有找到与输入规章A内容最为相似的规章B。
********************
file2_194_第 121.539 条_(a)
********************
file2_194_第 121.539 条_(b)
********************
file2_119_第 121.391 条_(f)
********************
file2_119_第 121.391 条_(f)
********************
file2_205_第 121.555 条_(b)
********************
file2_205_第 121.555 条_(c)

输入规章A主要强调的是在发现燃油不正常状态时，应立即采取措施，并报告飞行签派员，以确保剩余油量不低于最后储备燃油量。在规章B中，file2_205_第 121.555 条_(c)部分也提到了类似的燃油管理措施，即如果燃油检查结果显示可能低于特定油量，机长需要评估各种条件以确保安全着陆时的燃油量不低于最后储备燃油量。这与输入规

********************
未找到相似内容

输入规章A涉及的是飞行机组在特定情况下执行应急设备检查的要求，而规章B中的内容主要涉及仪表或设备失效、机组成员的应急生存训练、紧急情况下的运行程序等，没有直接与规章A中提到的“飞行机组值勤期内首次进入飞机驾驶舱，或飞行机组发生更换，或全部机组离开航空器并再次进入时执行应急设备检查”相似的内容。
********************
未找到相似内容

输入规章A涉及的是客舱机组值勤期内进行应急设备检查的要求，而规章B中的内容主要涉及应急设备的检验、机组成员的应急生存训练、飞行值勤期限制等，没有直接对应的内容提到值勤期内的应急设备检查。因此，根据提供的信息，没有找到与输入规章A内容最为相似的规章B条款。
********************
未找到相似内容

输入规章A涉及的是客舱应急设备的检查责任分配，而规章B中的内容主要涉及仪表或设备失效时的程序和紧急情况下的运行处理。这些内容与输入规章A的主题不太相似，因此没有找到直接相关的类似内容。
********************
未找到相似内容

输入规章A涉及的是机组成员在旅客登机前进行的安保检查，而规章B中的内容主要涉及机组成员的值勤要求、飞行期间的责任和职责，没有直接提到安保检查的相关内容。因此，根据提供的规章内容，没有找到与输入规章A内容最为相似的规章。
********************
file2_205_第 121.555 条_(b)

输入规章A主要涉及飞行机组成员对燃油加注过程的监控和确认，以及机长在特定情况下的签字确认。规章B中的file2_205_第 121.555 条_(b)部分也提到了机长确保机上剩余燃油量的要求，这与输入规章A中关于燃油监控的内容相似。虽然它们并不完全相同，但它们都涉及到燃油管理和机长的责任，因此这个选项是与输入规章A最相似的内容。其他选项则更多地涉及飞行中燃油管理的政策和程序，以及紧急情况下的燃油宣布，与输入规章A的内容不太相符。
********************
file2_194_第 121.539 条_(c)

输入规章A提到的是客舱乘务员根据手册和公司政策进行机上准备工作，而规章B中的file2_194_第 121.539 条_(c)提到的是机组成员的服务程序不得影响客舱乘务员履行安

In [116]:
len(top1_simi_result)

87

In [117]:
for key1, text1, key2, text2, simi_score in top1_simi_result:
    print(key1)
    print(text1)
    print(key2)
    print(text2)
    print(simi_score)
    print('------------------------------------------')

file1_4_2.3.
如需偏离标准操作程序， 飞行机组必须进行充分的沟通， 包括与其他相关机组 人员、 地面人员以及空中交通管制员等的沟通， 并评估偏离标准操作程序可能 造成的风险。
file2_208_第 121.558 条_(b)
补充运行的紧急情况 在使用飞行跟踪系统实施运行控制的飞行期间， 合格 证持有人的相关管理人员发现需要其立即决断和处置的紧 急情况时， 应当将紧急情况通知机长， 确实弄清机长的决断， 并且应当将该决断作出记录。如果在上述情况下，该管理人 员不能与飞行人员取得联系，则应当宣布进入应急状态，并 采取他认为在此种情况下为保证飞行安全应当采取的任何 行动。
0.7524038057288005
------------------------------------------
file1_7_3.2.
飞行机组在运行过程中，必须使用公司配备的经批准的检查单；
file2_74_第 121.315 条_(c)
驾驶舱检查单 经批准的检查单应当放置在每架飞机驾驶舱内方便 飞行机组成员使用的地方，飞行机组在操作飞机时应当遵循 检查单规定的程序。
0.7564717334011755
------------------------------------------
file1_30_8.2.
在飞行的关键阶段， 全体机组成员不得从事飞机安全运行所必需的工作之外的 任何其他工作，机组任何成员也不得承担这些工作。预定厨房供应品，确认旅 客的衔接航班，对旅客进行公司的广告宣传，介绍风景名胜的广播，填写与运 行无关的公司报告表、记录表等工作都不是飞机安全运行所必需的工作。
file2_194_第 121.539 条_(a)
机组成员的值勤要求 在飞行的关键阶段， 合格证持有人不得要求飞行机组 成员完成飞机安全运行所必需的工作之外的任何其他工作， 飞行机组任何成员也不得承担这些工作。预定厨房供应品， 确认旅客的衔接航班，对旅客进行合格证持有人的广告宣 传，介绍风景名胜的广播，填写与运行无关的公司报告表、 记录表等工作都不是飞机安全运行所必需的工作。
0.7977028009516467
------------------------------------------
file1_31_8.3.
在飞行关键阶段，飞行机组成员不得从事可能分散飞行机组其